In [2]:
from data import load_dataset, load_dataset_session
import numpy as np
import os
import pandas as pd
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, 'data/ml-1m/')

#### User_dict, User_history npy 파일 만들기

In [2]:
convert = pd.read_csv(os.path.join(DATA_DIR, 'ratings_converted.csv'))
convert = convert.applymap(int)
convert = convert.sort_values(by='timestamp', ascending=True)       #이렇게 정리해야 user_dict의 history가 최근순으로 정렬됨? -> 근데 어차피 일관성만 있으면 괜찮을 듯
convert

,userID,itemID,ratings,timestamp
268822,6038,662,4,956703932
88596,6038,489,5,956703954
655460,6038,1715,4,956703954
29800,6038,1403,4,956703977
366450,6038,1453,5,956703977
...,...,...,...,...
415827,5946,2260,4,1046437932
174096,5310,2376,4,1046444711
264155,4956,2506,4,1046454320
371493,4956,1027,5,1046454443


In [3]:
users_dict = {user : [] for user in set(convert["userID"])}

In [4]:
# add (movie, rating) pair to the user dictionary
## add movie rated >= 4 only
convert_gen = convert.iterrows()
users_dict_for_history_len = {user : [] for user in set(convert["userID"])}
for data in convert_gen:
    users_dict[data[1]['userID']].append((data[1]['itemID'], data[1]['ratings']))
    if data[1]['ratings'] >= 4:
        users_dict_for_history_len[data[1]['userID']].append((data[1]['itemID'], data[1]['ratings']))

In [5]:
users_history_lens = [len(users_dict_for_history_len[u]) for u in set(convert["userID"])]

In [6]:
# save each data as npy to save time for future evaluation
np.save("./data/final_user_dict.npy", users_dict)
np.save("./data/final_users_history_len.npy", users_history_lens)

#### load_dataset 함수 확인

In [2]:
train_users_num, train_items_num, train_users_dict, train_users_history_lens, movies_id_to_movies = load_dataset(DATA_DIR, 'train')

Interact Data loading...
Data loading complete!
Data preprocessing...
total users_num : 6039, total items_num : 2819
train_users_num : 4831
Done


In [3]:
eval_users_num, _, test_users_dict, test_users_history_lens, _ = load_dataset(DATA_DIR, 'test')

Interact Data loading...
Data loading complete!
Data preprocessing...
total users_num : 6039, total items_num : 2819
eval_users_num : 1208
Done


In [4]:
# key 값 겹치는지 확인
train_key = train_users_dict.keys()
test_key = test_users_dict.keys()
print(train_key & test_key)
print(len(train_key), len(test_key))

set()
4831 1208


In [5]:
# history_len에 positive rating만 들어간건지 확인
for user_id, history in train_users_dict.items():
    cnt = 0
    for movie_id, rating in history:
        if rating > 3:
            cnt += 1
    
    if cnt != train_users_history_lens[user_id]:
        print(user_id, cnt, train_users_history_lens[user_id])

In [7]:
for user_id, history in test_users_dict.items():
    cnt = 0
    for movie_id, rating in history:
        if rating > 3:
            cnt += 1
    
    if cnt != test_users_history_lens[user_id]:
        print(user_id, cnt, test_users_history_lens[user_id])

#### load_dataset_session 함수 확인

In [10]:
users_num, items_num, train_users_dict, train_users_history_lens, movies_id_to_movies = load_dataset_session(DATA_DIR, 'train')
users_num, items_num, eval_users_dict, eval_users_history_lens, movies_id_to_movies = load_dataset_session(DATA_DIR, 'eval')

Interact Data loading...
Data loading complete!
Data preprocessing...
total users_num : 6039, total items_num : 2819
Interact Data loading...
Data loading complete!
Data preprocessing...
total users_num : 6039, total items_num : 2819


In [11]:
len(train_users_dict), len(eval_users_dict)

(6039, 6039)

In [12]:
for idx, ((user_id, history), (user_id2, history2)) in enumerate(zip(train_users_dict.items(), eval_users_dict.items())):
    print(len(history), len(history2), len(history)/(len(history)+len(history2)))
    if idx == 20: break

38 9 0.8085106382978723
84 20 0.8076923076923077
36 9 0.8
14 3 0.8235294117647058
104 26 0.8
50 12 0.8064516129032258
24 6 0.8
99 24 0.8048780487804879
72 17 0.8089887640449438
257 64 0.8006230529595015
73 18 0.8021978021978022
16 4 0.8
80 19 0.8080808080808081
15 3 0.8333333333333334
130 32 0.8024691358024691
16 3 0.8421052631578947
138 34 0.8023255813953488
184 46 0.8
156 38 0.8041237113402062
18 4 0.8181818181818182
12 3 0.8


In [13]:
_, _, prev_train_users_dict, _, _ = load_dataset(DATA_DIR, 'train')

Interact Data loading...
Data loading complete!
Data preprocessing...
total users_num : 6039, total items_num : 2819
train_users_num : 4831
Done


In [14]:
for idx, ((user_id, history), (user_id2, history2)) in enumerate(zip(eval_users_dict.items(), prev_train_users_dict.items())):
    print(history)
    print(history2)
    if idx == 20: break

[(2321, 4), (934, 5), (1243, 4), (763, 5), (1679, 3), (2460, 4), (2046, 5), (877, 5), (214, 4)]
[(2321, 4), (934, 5), (1243, 4), (763, 5), (1679, 3), (2460, 4), (2046, 5), (877, 5), (214, 4), (698, 4), (498, 4), (1950, 4), (1403, 5), (1460, 5), (2266, 5), (708, 4), (1404, 4), (774, 5), (767, 5), (1452, 4), (125, 5), (814, 4), (696, 3), (2040, 4), (945, 5), (2009, 4), (918, 4), (528, 3), (2128, 4), (437, 4), (2273, 4), (2034, 4), (768, 5), (1661, 3), (490, 4), (880, 3), (1945, 3), (484, 4), (491, 5), (582, 3), (0, 5), (1641, 4), (1691, 5), (611, 4), (1133, 4), (1364, 4), (43, 5)]
[(881, 4), (896, 3), (890, 4), (1973, 3), (950, 5), (2147, 4), (903, 5), (877, 5), (256, 5), (2078, 4), (489, 5), (424, 5), (1336, 4), (1809, 5), (1486, 5), (95, 5), (2294, 5), (919, 5), (2266, 4), (992, 5)]
[(881, 4), (896, 3), (890, 4), (1973, 3), (950, 5), (2147, 4), (903, 5), (877, 5), (256, 5), (2078, 4), (489, 5), (424, 5), (1336, 4), (1809, 5), (1486, 5), (95, 5), (2294, 5), (919, 5), (2266, 4), (992, 5)

In [17]:
# history_lens를 그럼 train, eval마다 다르게 가져가야함
# add (movie, rating) pair to the user dictionary
## add movie rated >= 4 only
train_users_history_lens = []
for id, history in train_users_dict.items():
    cnt = 0
    for his in history:
        if his[1] >= 4:
            cnt += 1
    train_users_history_lens.append(cnt)

eval_users_history_lens = []
for id, history in eval_users_dict.items():
    cnt = 0
    for his in history:
        if his[1] >= 4:
            cnt += 1
    eval_users_history_lens.append(cnt)

In [19]:
len(train_users_history_lens), len(eval_users_history_lens)
eval_users_history_lens[:10]

[8, 18, 6, 2, 18, 8, 5, 22, 11, 53]

In [20]:
np.save("./data/final_train_users_history_len.npy", train_users_history_lens)
np.save("./data/final_eval_users_history_len.npy", eval_users_history_lens)